In [191]:
# Libraries for data loading, data manipulation and data visulisation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import chardet # To provide a best estimate of the encoding that was used in the text data
import io # For string operations
%matplotlib inline

# Libraries for data preparation and model building
from imblearn.over_sampling import RandomOverSampler
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, TweetTokenizer, TreebankWordTokenizer
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import math
import re
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from scipy.sparse import vstack
from scipy.sparse import csr_matrix
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn import feature_selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, log_loss
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [84]:
from google.colab import drive
drive.mount('/content/drive')
df_train = pd.read_csv('/content/drive/MyDrive/Professionele ontwikkeling/Data Science/Explore Data Science Course/Sprint 6_Advanced Classification/Predict/advanced-classification-predict/data/train.csv')
pd.set_option('display.max_colwidth', None)
df_train.head(10)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,sentiment,message,tweetid
0,1,"PolySciMajor EPA chief doesn't think carbon dioxide is main cause of global warming and.. wait, what!? https://t.co/yeLvcEFXkC via @mashable",625221
1,1,It's not like we lack evidence of anthropogenic global warming,126103
2,2,RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year in the war on climate change https://t.co/44wOTxTLcD,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, sexist, climate change denying bigot is leading in the polls. #ElectionNight",466954
5,1,Worth a read whether you do or don't believe in climate change https://t.co/ggLZVNYjun https://t.co/7AFE2mAH8j,425577
6,1,RT @thenation: Mike Pence doesn’t believe in global warming or that smoking causes lung cancer. https://t.co/gvWYaauU8R,294933
7,1,"RT @makeandmendlife: Six big things we can ALL do today to fight climate change, or how to be a climate activistÃ¢â‚¬Â¦ https://t.co/TYMLu6DbNM hÃ¢â‚¬Â¦",992717
8,1,"@AceofSpadesHQ My 8yo nephew is inconsolable. He wants to die of old age like me, but will perish in the fiery hellscape of climate change.",664510
9,1,RT @paigetweedy: no offense… but like… how do you just not believe… in global warming………,260471


In [86]:
# Step 1: Split into train / test
X_train, X_test, y_train, y_test = train_test_split(df_train['message'], df_train['sentiment'], test_size=0.15, random_state=42)
X_train.head()

6660     RT @sean_spicier: You know they're going to ruin it w/all the, 'What the eclipse tells us about climate change' hysteria in the aftermath\n#…
6867       RT @JamesHibberd: .@GameOfThrones star warns: Winter isn’t coming, climate change ‘terrifying’ https://t.co/uPckEiACT9 #GoTS7 https://t.co…
11268      RT @campaignsarah: 'Last year's Paris Agreement showed the world was united in its concern about climate change...' https://t.co/r28msde9LO
4012        RT @TheRoadbeer: After being stuck in my mentions for 24 hours about man made climate change, I pop out to find a conv about thigh-highs.…
14792     RT @hannahasbury_: I can comfortably wear shorts and a hoodie outside on a February morning but climate change isn't real or anything u know
Name: message, dtype: object

In [ ]:
class SampleSelector(BaseEstimator, TransformerMixin):
    def __init__(self, sample_percentage=1.0, stratify='y', random_state=42):
        self.sample_percentage = sample_percentage
        self.stratify = stratify
        self.random_state = random_state

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Set random seed for reproducibility
        np.random.seed(self.random_state)

        if self.stratify is not None and y is not None:
            # Determine the unique classes in y and their frequencies
            unique_classes, class_counts = np.unique(y, return_counts=True)

            # Determine the number of samples to select for each class
            num_samples_per_class = (class_counts * self.sample_percentage).astype(int)

            # Initialize an empty list to store the sampled indices
            sampled_indices = []

            # Iterate over each unique class
            for cls, num_samples in zip(unique_classes, num_samples_per_class):
                # Get the indices of samples belonging to the current class
                class_indices = np.where(y == cls)[0]

                # Randomly select samples from the current class
                sampled_indices.extend(np.random.choice(class_indices, size=num_samples, replace=False))
        else:
            # Determine the number of samples to select
            num_samples = int(len(X) * self.sample_percentage)

            # Randomly select samples
            sampled_indices = np.random.choice(len(X), size=num_samples, replace=False)

        # Return the randomly sampled subset
        return X.iloc[sampled_indices]

In [ ]:
class NoiseRemover(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'   # Find all hyperlinks
        subs_url = r''
        X_transformed = X.replace(to_replace = pattern_url, value = subs_url, regex = True)
        return X_transformed

In [ ]:
class EmoticonConverter(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        emoticon_dictionary = {':\)': 'smiley_face_emoticon',
                               ':\(': 'frowning_face_emoticon',
                               ':D': 'grinning_face_emoticon',
                               ':P': 'sticking_out_tongue_emoticon',
                               ';\)': 'winking_face_emoticon',
                               ':o': 'surprised_face_emoticon',
                               ':\|': 'neutral_face_emoticon',
                               ':\'\)': 'tears_of_joy_emoticon',
                               ':\'\(': 'crying_face_emoticon'}
        X_transformed = X.replace(emoticon_dictionary, regex=True)
        return X_transformed

In [ ]:
class PunctuationRemover(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        def expand_contractions(text):
            contractions = {"'t": " not","'s": " is","'re": " are","'ll": " will", "'m": " am"}
            pattern = re.compile(r"\b(" + "|".join(re.escape(key) for key in contractions.keys()) + r")\b")
            text = re.sub(r"n't\b", " not", text) # Replace "n't" with " not"
            text = pattern.sub(lambda match: contractions[match.group(0)], text) # Replace all other contractions except for "n't"
            return text

        def remove_punctuation(text):
            return ''.join([l for l in text if l not in string.punctuation])

        X_transformed = X.apply(expand_contractions)
        X_transformed = X_transformed.apply(remove_punctuation)

        return X_transformed

In [ ]:
class Tokenizer(BaseEstimator, TransformerMixin):
    def __init__(self, type='TweetTokenizer'):
        self.type = type

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        if self.type == 'TweetTokenizer':
            tokenizer = TweetTokenizer()
        else:
            tokenizer = TreebankWordTokenizer()
        X_transformed = X.apply(tokenizer.tokenize)
        return X_transformed


In [ ]:
class StopwordRemover(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        stop_words = set(stopwords.words('english'))
        # Remove stopwords using a vectorized operation
        X_transformed = X.apply(lambda tokens: [t for t in tokens if t.lower() not in stop_words])
        return X_transformed




In [138]:
class Lemmatizer(BaseEstimator, TransformerMixin):
    def __init__(self, pos='v'):
        self.pos = pos

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        lemmatizer = WordNetLemmatizer()
        X_transformed = X.apply(lambda tokens: [lemmatizer.lemmatize(word, pos=self.pos) for word in tokens])
        return X_transformed

t = Tokenizer()
b = t.transform(X_test)
sw = StopwordRemover()
a = sw.transform(b)
l = Lemmatizer()
c = l.transform(a)
print(c)
print(X_test[9059])

9059     [RT, @AntonioParis, :, Texas, experience, third, ‘, 500, -, year, ’, flood, 3, years, ., Bad, luck, climate, change, ?, think, ?, https://t.co/…]
169                                   [Conservative, columnist, siege, N, ., ., Times, debut, climate, change, ', via, FOX, NEWS, https://t.co/nHL8MNOmDa]
5331              [muslim, clerk, give, us, lecture, today, &, say, that'because, people, abandon, faith, rain, yet, ', !, !, dude, fuck, climate, change]
6333                            [Doomsday, forecasters, change, narrative, come, ice, age, ,, global, warm, climate, change, ..., finally, pick, scenario]
10383                                                                                       [Man, get, global, warm, freezer, fuck, food, ., cba, life, .]
                                                                               ...                                                                        
2802                                              [RT, @StephenSchlege

In [168]:
class Vectorize(BaseEstimator, TransformerMixin):
    def __init__(self, type='tfidf', max_df=1, min_df=1, ngram_range=(1,2), max_features=None):
        self.type = type
        self.max_df = max_df
        self.min_df = min_df
        self.ngram_range = ngram_range
        self.max_features = max_features

        if self.type == 'count':
            self.vectorizer = CountVectorizer(max_features=self.max_features, lowercase=True,
                                              max_df=self.max_df, min_df=self.min_df,
                                              ngram_range=self.ngram_range)
        elif self.type == 'tfidf':
            self.vectorizer = TfidfVectorizer(max_features=self.max_features, lowercase=True,
                                              max_df=self.max_df, min_df=self.min_df,
                                              ngram_range=self.ngram_range)
        else:
            raise ValueError("Invalid vectorizer type. Choose either 'count' or 'tfidf'")

    def fit(self, X, y=None):
        X_joined = [' '.join(tokens) for tokens in X]
        self.vectorizer.fit(X_joined)
        return self

    def transform(self, X, y=None):
        X_joined = [' '.join(tokens) for tokens in X]
        return self.vectorizer.transform(X_joined)

t = Tokenizer()
b = t.transform(X_test)
sw = StopwordRemover()
a = sw.transform(b)
l = Lemmatizer()
c = l.transform(a)
vect = Vectorize(type='tfidf', max_df=0.1, min_df=1, ngram_range = (1,2), max_features=10)
vect.fit(c)
z = vect.transform(c)
z


<2373x10 sparse matrix of type '<class 'numpy.float64'>'
	with 1300 stored elements in Compressed Sparse Row format>

In [195]:
class Resampler(BaseEstimator, TransformerMixin):
    def __init__(self, technique='balanced', random_state=42):
        self.technique = technique
        self.random_state = random_state

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        if self.technique == 'balanced':
            class_labels, counts = np.unique(y, return_counts=True)
            majority_count = max(counts)
            n_classes = len(class_labels)
            balanced_freq = X.shape[0] / n_classes  # Get the number of samples from the shape of X
            X_resampled, y_resampled = [], []
            for class_label, count in zip(class_labels, counts):
                indices = np.atleast_1d(np.where(y == class_label)[0])
                if count > balanced_freq:
                    resampled_indices = resample(indices, replace=False, n_samples=int(balanced_freq), random_state=self.random_state)
                elif count < balanced_freq:
                    resampled_indices = resample(indices, replace=True, n_samples=int(balanced_freq), random_state=self.random_state)
                else:
                    resampled_indices = indices
                if len(resampled_indices) > 0:
                    resampled_X = X[resampled_indices]
                    X_resampled.append(resampled_X)
                    y_resampled.extend([class_label] * resampled_X.shape[0])
            if X_resampled:
                X_resampled = vstack(X_resampled)
            else:
                X_resampled = csr_matrix((0, X.shape[1]))  # Create an empty sparse matrix
            y_resampled = np.array(y_resampled)

        elif self.technique == 'smote':
            sampler = SMOTE(random_state=self.random_state)
            X_dense = X.toarray()
            X_resampled, y_resampled = sampler.fit_resample(X_dense, y)
            X_resampled = csr_matrix(X_resampled)

        else:
            raise ValueError("Invalid resampling technique.  Choose either 'random' or 'smote'")

        return X_resampled, y_resampled

        return X_resampled, y_resampled

t = Tokenizer()
b = t.transform(X_test)
sw = StopwordRemover()
a = sw.transform(b)
l = Lemmatizer()
c = l.transform(a)
vect = Vectorize(type='tfidf', max_df=0.1, min_df=1, ngram_range = (1,2), max_features=10)
vect.fit(c)
z = vect.transform(c)
resam = Resampler(technique='balanced')
resam.fit(z, y_test)
x, y = resam.transform(z, y_test)
print(x.shape)
print(y.shape)

(2372, 10)
(2372,)


In [176]:
class Scaler(BaseEstimator, TransformerMixin):
    def __init__(self, type='robust'):
        self.type = type

        if self.type == 'robust':
            self.scaler = RobustScaler(with_centering=False)
        elif self.type == 'minmax':
            self.scaler = MinMaxScaler(with_centering=False)
        elif self.type == 'maxabs':
            self.scaler = MaxAbsScaler(with_centering=False)
        else:
            raise ValueError("Invalid scaler type. Choose between 'robust', 'minmax' or 'maxabs'.")

    def fit(self, X, y=None):
        self.scaler.fit(X)
        return self

    def transform(self, X, y=None):
        return self.scaler.transform(X)

t = Tokenizer()
b = t.transform(X_test)
sw = StopwordRemover()
a = sw.transform(b)
l = Lemmatizer()
c = l.transform(a)
vect = Vectorize(type='tfidf', max_df=0.1, min_df=1, ngram_range = (1,2), max_features=10)
vect.fit(c)
z = vect.transform(c)
resam = Resampler(technique='balanced')
resam.fit(z, y_test)
x, y = resam.transform(z, y_test)
scal = Scaler(type='robust')
scal.fit(x)
result = scal.transform(x)
resu


<2373x10 sparse matrix of type '<class 'numpy.float64'>'
	with 1300 stored elements in Compressed Sparse Row format>

In [ ]:
'''
# Step 3: Define custom transformers

class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Apply text preprocessing steps (steps 4-9)
        X_processed = X.apply(lambda text: preprocess_text(text))
        return X_processed

def preprocess_text(text):
    # Implement text preprocessing steps (steps 4-9)
    # Example: remove hyperlinks, convert emoticons, remove punctuation, tokenize, remove stopwords, lemmatize
    # Return preprocessed text
    pass

# Step 10: Define pipeline for vectorization and feature selection
vectorizer = TfidfVectorizer()
feature_selector = SelectFromModel(estimator=RandomForestClassifier())

vectorization_pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('feature_selector', feature_selector)
])

# Step 11: Define pipeline for modeling
models = {
    'Logistic Regression': LogisticRegression(),
    'Support Vector Machine': SVC(),
    'Multinomial Naive Bayes': MultinomialNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

modeling_pipeline = Pipeline([
    ('model', models['Logistic Regression'])  # Placeholder, will be replaced during grid search
])

# Step 12: Define parameter grid for grid search
param_grid = {
    'model': list(models.values()),  # Models to try
    # Add parameters for each model if necessary
}

# Step 13: Perform grid search
grid_search = GridSearchCV(modeling_pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_resampled, y_train_resampled)

# Step 14: Evaluate best model on test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
'''


In [ ]:
# Define your preprocessing steps
preprocessing_steps = [
    ('sampling', RandomSampler()),
    # Add other preprocessing steps here
]

# Define your model
model = YourModelHere()

# Create the pipeline
pipeline = Pipeline(preprocessing_steps + [('model', model)])

# Define parameter grid for GridSearchCV
param_grid = {
    'sampling__sample_percentage': [0.5, 0.75, 1.0],  # Adjust the sampling percentages as needed
    # Add other hyperparameters for your model
}

# Create GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5)